In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from datetime import datetime, timedelta, date

from quantopian.pipeline import Pipeline
from quantopian.pipeline.data import Fundamentals as F
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.filters import StaticAssets
from quantopian.research import run_pipeline


DEBUG = True


BILLION = 1000000000

MARKER_INDEX = 0
COLOR_INDEX = 1
DASH_INDEX = 2

GMxPS = 'gross margin vs price-to-sales'

CLOSE_COL = 'close'

# colors: https://matplotlib.org/3.1.0/gallery/color/named_colors.html
ATTR = {
    'close': ('.', 'k', tuple()),
    'market_cap': ('.', 'k', tuple()),
    'shares': ('.', 'deeppink', tuple()),
    
    'revenue': ('s', 'orange', (2,2)),
    'gross_profit' : ('p', 'sandybrown', (2,2)),
    'operating_income' : ('v', 'lightblue', (2,2)),
    'net_income': ('o', 'b', (2,2)),
    
    'ps': ('s', 'orange', (2,2)),
    'pe': ('o', 'b', (2,2)),
    'pfcf': ('^', 'g', (2,2)),
    'pb': ('*', 'grey', (2,2)),
    
    'equity': ('*', 'grey', (2,2)),
    'cash' : ('+', 'g', (2,2)),
    'short_term_liabilities': ('>', 'red', (2,2)),
    'long_term_liabilities': ('<', 'darkred', (2,2)),
    
    
    'fcf': ('^', 'g', (2,2)),
}


def make_pipeline(tickers):
    mask = StaticAssets(tickers)
    
    close = USEquityPricing.close.latest
    market_cap = F.market_cap.latest
    
    # ratios
    ps = F.ps_ratio.latest
    pe = F.pe_ratio.latest
    pfcf = F.fcf_ratio.latest
    pb = F.pb_ratio.latest
    
    # income statement
    revenue = F.total_revenue.latest
    gross_profit = F.gross_profit.latest
    operating_income = F.operating_income.latest
    net_income = F.net_income_common_stockholders.latest
    
    gross_margin = F.gross_margin.latest
    revenue_growth = F.revenue_growth.latest
    
    ev = F.enterprise_value.latest
    
    operating_expense = F.operating_expense.latest
    
    # balance sheet statement
    equity = F.stockholders_equity.latest
    cash = F.cash_and_cash_equivalents.latest
    stl = F.current_liabilities.latest
    ltl = F.long_term_debt.latest
    
    fcf_yield = F.fcf_yield.latest
    
    ##equity = assets - stl - ltl
    
    # cash flow statement
    fcf = F.free_cash_flow.latest
    
    shares = F.shares_outstanding.latest
    
    period_ending_date = F.period_ending_date.latest
    
    pipe = Pipeline(
        columns={
            'close': close,
            'period_ending_date': period_ending_date,
            'market_cap': market_cap,
            'shares': shares,

            'revenue': revenue,
            'gross_profit' : gross_profit,
            'operating_income': operating_income,
            'net_income': net_income,
            
            'gross_margin': gross_margin,
            'revenue_growth': revenue_growth,
            
            'ps': ps,
            'pe': pe,
            'pfcf': pfcf,
            'pb': pb,
            
            'ev/sales': ev / revenue,
            
            'equity': equity,
            'cash' : cash,
            'short_term_liabilities': stl,
            'long_term_liabilities': ltl,
            
            'fcf': fcf,
            'fcf_yield': fcf_yield,
        },
        screen=mask
    )
    return pipe

 
def graph(pipeline, ax, columns, state):
    co = pipeline.reset_index()['level_1']
    x = pipeline[columns[0]]
    
    for column in columns[1:]:
        y = pipeline[column]
        ax.scatter(x, y)
        for xy in zip(x, y):
            ax.annotate(co, xy=xy, textcoords='data')
    ax.legend(columns, loc='upper left')


def get_columns(state):
    columns = []
    
    if state == GMxPS:
        columns += [        
            'gross_margin',
            'ps',
        ]
    return columns
    

OVERLAY_CLOSE = True

def main(tickers, graphs, start_date='2020-08-01'): 
    FIG_WIDTH = 15
    FIG_HEIGHT = len(tickers)*3

    yest = (datetime.now() - timedelta(days=1)).date()

    _pipeline = run_pipeline(make_pipeline(tickers), start_date, yest)

    #plt.subplots_adjust(hspace=.5)
    fig = plt.figure(figsize=(FIG_WIDTH, FIG_HEIGHT))

    pipeline = pd.DataFrame(_pipeline)
    
    charts = [
        ('ps', 'gross_margin'),
        ('ps', 'revenue_growth'),
        ('ev/sales', 'gross_margin'),
        ('ev/sales', 'revenue_growth'),
    ]

    for index, row in pipeline.iterrows():
        ticker = index[1]
        
        for i, chart in enumerate(charts):
            ycol, xcol = chart
            yval = row[ycol]
            xval = row[xcol]
            ax = fig.add_subplot(len(charts), 1, i+1)
        
            ax.scatter(xval, yval)
            ax.annotate(ticker, xy=(xval, yval), textcoords='data')
            ax.set_title('{0} vs {1}'.format(ycol, xcol))
            ax.set_xlabel(xcol)
            ax.set_ylabel(ycol)

            
tickers = symbols([
    'nflx',
    'mdb',
    'sq',
    'pypl',
    'shop',
    'eb',
    'snap',
    'docu',
    'tdoc'
    'upwk',
    'se',
    'etsy',
    'meli',
    'nvta',
    'ayx',
    'fsly',
    'crwd',
    'fvrr',
    'apps',
    'api',
    'roku',
    'isrg',
    #'spce',  # outlier
    'spot',
    'net',
    'chgg',
    'twlo',
    'ddog',
    'shop',
    'work',
    'wday',
    'splk',
    'tsla',
    'nflx',
    'yext',
    'fb',
    'uber',
    'lyft',
    'twtr',
    'okta',
    'apps',
    'z',
    'rdfn',
    'cldr',
    'crm',
    'pd',
    'zm',
    'dbx',
    'net',
    'bl',
    'zen',
    'znga',
    'pton',
    'snow',
])

start_date = (datetime.now() - timedelta(days=1)).date()
graphs = (GMxPS,)

main(tickers, graphs, start_date=start_date)


# 